In [14]:
# Importing libraries
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
from google_trans_new import google_translator  


In [3]:
# Obtaining data from IMDb
url = 'http://www.imdb.com/chart/top'
html = requests.get(url)
soup = BeautifulSoup(html.text, 'html.parser')

In [4]:
# Selecting data
films = soup.select('td.titleColumn')
crew = [a.attrs.get('title') for a in soup.select('td.titleColumn a')]
ratings = [b.attrs.get('data-value') for b in soup.select('td.posterColumn span[name=ir]')]

In [5]:
# Creating an empty dataframe
imdb = pd.DataFrame()

In [6]:
# Creating to empty lists - columns
title = []
year = []

# Split title text in Title and Year 
for i in range (0, len(films)):
    title_text = films[i].get_text()
    title_split = (' '.join(title_text.split()).replace('.', ''))
    titles = title_split[len(str(i))+1:-7]
    title.append(titles)
    years = re.search('\((.*?)\)', title_text).group(1)
    year.append(years)


In [7]:
# Adding columns to our dataframe
imdb['Title'] = title
imdb['Year'] = year
imdb['Starring'] = crew
imdb['Rate'] = ratings

In [8]:
imdb.head()

,Title,Year,Starring,Rate
0,Cadena perpetua,1994,"Frank Darabont (dir.), Tim Robbins, Morgan Fre...",9.219631568656466
1,El padrino,1972,"Francis Ford Coppola (dir.), Marlon Brando, Al...",9.147485565257945
2,El padrino: Parte II,1974,"Francis Ford Coppola (dir.), Al Pacino, Robert...",8.97987482883011
3,El caballero oscuro,2008,"Christopher Nolan (dir.), Christian Bale, Heat...",8.969640461405493
4,12 hombres sin piedad,1957,"Sidney Lumet (dir.), Henry Fonda, Lee J. Cobb",8.934645774547555


In [11]:
def translation(x):
    # Creating an object of the Translator class
    translator = google_translator()  
    # Translating to English
    translate_text = translator.translate(x ,lang_tgt='en')
    return translate_text

imdb['English_Title'] = imdb.Title.apply(translation) 

In [12]:
imdb.head()

,Title,Year,Starring,Rate,English_Title
0,Cadena perpetua,1994,"Frank Darabont (dir.), Tim Robbins, Morgan Fre...",9.219631568656466,Life imprisonment
1,El padrino,1972,"Francis Ford Coppola (dir.), Marlon Brando, Al...",9.147485565257945,The Godfather
2,El padrino: Parte II,1974,"Francis Ford Coppola (dir.), Al Pacino, Robert...",8.97987482883011,The godfather: part II
3,El caballero oscuro,2008,"Christopher Nolan (dir.), Christian Bale, Heat...",8.969640461405493,The dark knight
4,12 hombres sin piedad,1957,"Sidney Lumet (dir.), Henry Fonda, Lee J. Cobb",8.934645774547555,12 men without mercy


In [13]:
imdb.to_csv('IMDb.csv')